In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys

## Read data and construct confidence intervals

In [ ]:
alpha = 0.1
ns = [10000, 20000, 30000]
N_total = 167434
trials = 100
K = 3 # number of folds
theta_star = 0.2622 # computed on galaxy zoo 2

data_dir = 'data/cross-prediction-outputs/'

columns = ["lb","ub","coverage","estimator","n"]
temp_df = pd.DataFrame(np.zeros((3,len(columns))), columns=columns)
    
results = []

for n in ns:
    
    n_val = int(n*0.9)
    n_tr = n - n_val
    N = N_total - n
    fold_size = n//K
    
    for i in range(trials):
        

        Y = np.zeros(n)
        Yhat = np.zeros(n)
        Yhat_unlabeled = np.zeros(N)

        for k in range(K):
            filename = data_dir + f"crosspredict_{i}_{n}_{k}.npz"
            data_temp = np.load(filename)
            Y[k*fold_size:(k+1)*fold_size] = data_temp['labels_val']
            Yhat[k*fold_size:(k+1)*fold_size] = data_temp['preds_val']
            Yhat_unlabeled += data_temp['preds_unlabeled'] / K

        point_est = Yhat_unlabeled.mean() + (Y - Yhat).mean()
        halfwidth = norm.ppf(1-alpha/2) * np.sqrt(np.var(Yhat_unlabeled)/N + np.var(Y - Yhat)/n)
        cp_ci_l = point_est - halfwidth
        cp_ci_u = point_est + halfwidth
        
        temp_df.loc[0] = cp_ci_l, cp_ci_u, (cp_ci_l <= theta_star) & (theta_star <= cp_ci_u), "cross-prediction", n
        

        classical_point_est = Y.mean()
        classical_halfwidth = norm.ppf(1-alpha/2) * np.sqrt(np.var(Y)/n)
        classical_ci_l = classical_point_est - classical_halfwidth
        classical_ci_u = classical_point_est + classical_halfwidth

        temp_df.loc[1] = classical_ci_l, classical_ci_u, (classical_ci_l <= theta_star) & (theta_star <= classical_ci_u), "classical", n


        
        filename = data_dir + f"ppi_{i}_{n}.npz"
        data_temp = np.load(filename)
        Y = data_temp['labels_val']
        Yhat = data_temp['preds_val']
        Yhat_unlabeled = data_temp['preds_unlabeled']

        point_est = Yhat_unlabeled.mean() + (Y - Yhat).mean()
        halfwidth = norm.ppf(1-alpha/2) * np.sqrt(np.var(Yhat_unlabeled)/N + np.var(Y - Yhat)/n_val)
        ppi_ci_l = point_est - halfwidth
        ppi_ci_u = point_est + halfwidth
        

        temp_df.loc[2] = ppi_ci_l, ppi_ci_u, (ppi_ci_l <= theta_star) & (theta_star <= ppi_ci_u), "PPI", n
        
        results += [temp_df.copy()]
final_df = pd.concat(results,ignore_index=True)
final_df["width"] = final_df["ub"] - final_df["lb"]

## Plot results

In [ ]:
col = np.array([sns.color_palette("Set2")[1], sns.color_palette("Set2")[2], sns.color_palette("Set2")[0]])
sns.set_theme(font_scale=1.4, style='white', palette=col, rc={'lines.linewidth': 3})
# defaults = {'n': 100, 'beta': 0}
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3.3))
sns.lineplot(ax=axs[0],data=final_df, x='n', y='coverage', hue='estimator', alpha=0.9, errorbar=None, marker="*", markersize=14)
sns.lineplot(ax=axs[1],data=final_df, x='n', y='width', hue='estimator', alpha=0.9, marker="*", markersize=14)

axs[0].axhline(1-alpha, color="#888888", linestyle='dashed', zorder=1, alpha=0.9)
handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles=handles, labels=labels)
axs[0].get_legend().remove()
axs[0].set_ylim([0.5,1])

axs[0].lines[0].set_linestyle("--")
axs[1].lines[0].set_linestyle("--")
axs[0].lines[1].set_linestyle("--")
axs[1].lines[1].set_linestyle("--")
axs[0].lines[2].set_linestyle("--")
axs[1].lines[2].set_linestyle("--")

sns.despine(top=True, right=True)
plt.tight_layout()

# save plot
os.makedirs('./galaxies_results/', exist_ok=True)
plt.savefig('./galaxies_results/galaxy_comparison.pdf')
plt.show()